In [1]:
#Импортируем нужные библиотеки
import pandas as pd
import numpy as np

#Алгоритм бустинга, идея в том, что мы берем много не очень сильных алгоритмов и усредняем спец образом
import xgboost as xgb

#Библиотека для автоматического подбора параметров

import hyperopt as hp
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#Многие алгоритмы имеют элемент псевдослучайности - фиксируем эту случайность, чтобы результаты были воспроизводимы
SEED = 13
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def first_processing_obj(df, main_df=None):
    #df - датафрейм, который обрабатывается
    #main_df - датафрейм, на основании когторого обрабатываем, пропуски на тесте заполянем за счет трейна
    if main_df is None:
        main_df = df
    #Возраст в годах 
    #Что можно: разбить на бины ( от 10 до 20, от 20 до 30 итд)
    df['age_year'] = df.age / 365
    #LabelПризнаки приведем к 0..n
    df['sex'] = np.where(df.gender == 2, 0, 1)
    df.drop(['id', 'alco', 'active', 'smoke', 'age', 'gender'], axis=1, inplace=True)
    rcols = ['height', 'weight', 'ap_hi', 'ap_lo', 'age_year']
    
    #Генерируем небольшие фичи
    df['ap_hi_200'] = df.ap_hi - 200
    df['por_al'] = round(df.ap_lo / df.ap_hi, 2)
    df['por_weight'] = round(df.height / df.weight, 2)
    df['mean_'+ 'por_al'] = round(df['por_al'] - df['por_al'].mean(),2)
    df['mean_'+ 'por_weight'] = round(df['por_weight'] - df['por_weight'].mean(),2)
    for col in tqdm(rcols):
        if col!='ap_lo':
            df['log_'+col] = round(np.log(df[col]),2)
        df['mean_'+col] = round(df[col] - df[col].mean(),2)
        df[col] = round(df[col], 2)
        for col_2 in rcols:
            if col != col_2:
                df[col+'+'+col_2] = df[col]+df[col_2]
                df[col+'-'+col_2] = df[col]-df[col_2]
    df['bad_mean_diff_heigh'] = round(df.height - main_df[main_df.cardio==0].height.mean(), 2)
    df['good_mean_diff_heigh'] = round(df.height - main_df[main_df.cardio==1].height.mean(), 2)
    df['bad_mean_diff_weight'] = round(df.weight - main_df[main_df.cardio==0].weight.mean(), 2)
    df['good_mean_diff_weight'] = round(df.weight - main_df[main_df.cardio==1].weight.mean(), 2)
    df['bad_mean_diff_ap_hi'] = round(df.ap_hi - main_df[main_df.cardio==0].ap_hi.mean(), 2)
    df['good_mean_diff_ap_hi'] = round(df.ap_hi - main_df[main_df.cardio==1].ap_hi.mean(), 2)
    df['bad_mean_diff_ap_lo'] = round(df.ap_lo - main_df[main_df.cardio==0].ap_lo.mean(), 2)
    df['good_mean_diff_ap_lo'] = round(df.ap_lo - main_df[main_df.cardio==1].ap_lo.mean(), 2)
    df['bad_mean_diff_frac_pressure'] = round(df.por_al - main_df[main_df.cardio==0].por_al.mean(), 2)
    df['good_mean_diff_frac_pressure'] = round(df.por_al - main_df[main_df.cardio==1].por_al.mean(), 2)
    df['bad_mean_diff_frac_hw'] = round(df.por_weight - main_df[main_df.cardio==0].por_weight.mean(), 2)
    df['good_mean_diff_frac_hw'] = round(df.por_weight - main_df[main_df.cardio==1].por_weight.mean(), 2)
    df['chol_1'] = np.where(df.cholesterol == 1, 1, 0)
    df['chol_2'] = np.where(df.cholesterol == 2, 1, 0)
    df['chol_3'] = np.where(df.cholesterol == 3, 1, 0)
    df['gl_1'] = np.where(df.gluc == 1, 1, 0)
    df['gl_2'] = np.where(df.gluc == 2, 1, 0)
    df['gl_3'] = np.where(df.gluc == 3, 1, 0)
    df.drop(['cholesterol', 'gluc'], axis=1, inplace=True)
    df.fillna(0, inplace=True)
    return df    

def first_scler_obj(df, stdScaler = None, mmScaler = None):
    #Давайте посмотрим какая нормализация лучше - выполним и ту и ту
    if stdScaler is None:
        stdScaler = StandardScaler()
        scaled_std = pd.DataFrame(stdScaler.fit_transform(df.drop(['chol_1', 'chol_2', 'chol_3', 'gl_1', 'gl_2', 'gl_3','sex'], axis=1).values),
                          columns=[i+'_std' for i in df.drop(['chol_1', 'chol_2', 'chol_3', 'gl_1', 'gl_2', 'gl_3','sex'], axis=1).columns])
    else:
        scaled_std = pd.DataFrame(stdScaler.transform(df.drop(['chol_1', 'chol_2', 'chol_3', 'gl_1', 'gl_2', 'gl_3','sex'], axis=1).values),
                          columns=[i+'_std' for i in df.drop(['chol_1', 'chol_2', 'chol_3', 'gl_1', 'gl_2', 'gl_3','sex'], axis=1).columns])
    if mmScaler is None:
        mmScaler = MinMaxScaler()
        scaled_mm = pd.DataFrame(mmScaler.fit_transform(df.drop(['chol_1', 'chol_2', 'chol_3', 'gl_1', 'gl_2', 'gl_3','sex'], axis=1).values),
                          columns=[i+'_mm' for i in df.drop(['chol_1', 'chol_2', 'chol_3', 'gl_1', 'gl_2', 'gl_3','sex'], axis=1).columns])
    else:
        scaled_mm = pd.DataFrame(mmScaler.transform(df.drop(['chol_1', 'chol_2', 'chol_3', 'gl_1', 'gl_2', 'gl_3','sex'], axis=1).values),
                          columns=[i+'_mm' for i in df.drop(['chol_1', 'chol_2', 'chol_3', 'gl_1', 'gl_2', 'gl_3','sex'], axis=1).columns])
    return stdScaler, mmScaler, pd.concat([scaled_mm, scaled_std, df[['sex', 'chol_1', 'chol_2', 'chol_3',
                                               'gl_1','gl_2','gl_3']]], axis=1)

def bad_data_replacer(df):
    #Ручная чистилка данных
    for i in tqdm(range(len(df))):
        if df.loc[i, 'ap_hi'] == 906:
                df.loc[i, 'ap_lo'] = 60
                df.loc[i, 'ap_hi'] = 90
        if df.loc[i, 'ap_lo'] <= 40:
            if df.loc[i, 'ap_lo']==20 or df.loc[i, 'ap_lo']==30 or df.loc[i, 'ap_lo'] == 0:
                df.loc[i, 'ap_lo'] = 80
            elif df.loc[i, 'ap_lo']>0 and df.loc[i, 'ap_lo']<=10:
                df.loc[i, 'ap_lo'] = df.loc[i, 'ap_lo']*10
            elif df.loc[i, 'ap_lo'] < 0:
                df.loc[i, 'ap_lo'] = df.loc[i, 'ap_lo'] * -1
            else:
                df.loc[i, 'ap_lo'] = df.loc[i, 'ap_hi'] - 50
        if df.loc[i, 'ap_lo'] > 200:
            df.loc[i, 'ap_lo'] = df.loc[i, 'ap_lo'] /10
        
        if df.loc[i, 'ap_hi'] > 0 and df.loc[i, 'ap_hi'] <= 25:
            df.loc[i, 'ap_hi'] = df.loc[i, 'ap_hi'] * 10
        if df.loc[i, 'ap_hi'] < 0:
            df.loc[i, 'ap_hi'] = df.loc[i, 'ap_hi'] * -1
        if df.loc[i, 'ap_hi'] > 250:
            if df.loc[i, 'ap_hi'] < 10000:
                df.loc[i, 'ap_hi'] = df.loc[i, 'ap_hi'] / 10
            else:
                df.loc[i, 'ap_hi'] = df.loc[i, 'ap_hi'] /100
    return df
        

In [3]:
data = pd.read_csv('ml5/train.csv', delimiter=';')
data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
data = bad_data_replacer(data)
train = first_processing_obj(data)
main_df = train.copy()
label = train.cardio
train.drop(['cardio'], axis=1, inplace=True)
stdScaler, mmScaler, train = first_scler_obj(train)

#Это бейзлайн, но вам надо оценивать качество - сделайте валидацию!
base_xgb = xgb.XGBClassifier(subsample=0.5, max_depth=3, gamma=0.5, learning_rate=0.1, reg_alpha=0, reg_lambda=0, min_child_weight=1, n_estimators=100)
base_xgb.fit(train, label)

100%|██████████| 5/5 [00:00<00:00, 65.01it/s]


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.5, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.5)

In [5]:
test = pd.read_csv('ml5/test.csv', delimiter=';')
test = bad_data_replacer(test)
test = first_processing_obj(test, main_df)
_, _, test = first_scler_obj(test, stdScaler=stdScaler, mmScaler=mmScaler)
baseline = pd.DataFrame(base_xgb.predict_proba(test))
baseline[1].to_csv('baseline16061400.csv', header=False, index=False)

100%|██████████| 5/5 [00:00<00:00, 84.58it/s]
